In [16]:
import fitz  # PyMuPDF para PDF
import docx
import os

def extract_text_from_pdf(pdf_path):
    """Extrae texto de un archivo PDF."""
    with fitz.open(pdf_path) as doc:
        return "\n".join([page.get_text("text") for page in doc])

def extract_text_from_docx(docx_path):
    """Extrae texto de un archivo DOCX."""
    doc = docx.Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_txt(txt_path):
    """Lee texto desde un archivo TXT."""
    with open(txt_path, "r", encoding="utf-8") as file:
        return file.read()

def extract_text(file_path):
    """Detecta el tipo de archivo y extrae su contenido."""
    _, ext = os.path.splitext(file_path)
    if ext.lower() == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext.lower() == ".docx":
        return extract_text_from_docx(file_path)
    elif ext.lower() == ".txt":
        return extract_text_from_txt(file_path)
    else:
        raise ValueError("Formato de archivo no soportado. Usa PDF, DOCX o TXT.")

# Prueba con un archivo
if __name__ == "__main__":
    file_path = "../01.01 - Análisis y diseño de algoritmos (2024-2) (1).pdf"  # Cambia esto con un archivo de prueba
    text = extract_text(file_path)
    print(text)  # Muestra los primeros 500 caracteres del contenido


Análisis y diseño de algoritmos
(3009430)

Objetivo del curso (oficial)
Cualquiera con una experiencia mínima en algoritmos y 
programación sabe que para un mismo problema pueden 
existir innumerables algoritmos de solución. Tales 
soluciones pueden variar en diversos aspectos, siendo de 
particular interés la eficiencia en términos de su costo 
computacional. 
En el contexto de dicha eficiencia se puede hablar de 
algoritmos "mejores" que otros, siendo el objetivo principal 
de esta asignatura brindarle elementos al estudiante para 
analizar y diseñar algoritmos eficientes.

•
Ofertar una asignatura disciplinar optativa para 
Ingeniería de Sistemas e Informática. De igual manera 
que sirva en el componente electivo de otros 
programas de la Universidad Nacional de Colombia.
•
Brindar un espacio para que los estudiantes a quienes 
les guste la programación mejoren sus habilidades
•
Preparar a los estudiantes para participar en el circuito 
colombiano de  maratones de programación y dem

In [ ]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Cargar la API key desde .env
load_dotenv()
genai.configure(api_key=os.getenv("API_KEY"))

In [23]:
def generate_content(prompt):
    """Genera contenido con Gemini a partir de un prompt."""
    model = genai.GenerativeModel("gemini-2.0-flash-001")
    response = model.generate_content(prompt)
    return response.text

def generate_study_materials(course_text):
    """Genera notas de clase, problemas, preguntas, objetivos y lecturas con Gemini."""
    
    prompts = {
        "notas_clase": f"Genera notas de clase detalladas basadas en este programa de curso:\n{course_text}",
        "problemas_practica": f"Genera problemas de práctica con soluciones basados en este programa de curso:\n{course_text}",
        "preguntas_discusion": f"Genera preguntas para discusión relacionadas con los temas de este programa de curso:\n{course_text}",
        "objetivos_aprendizaje": f"Genera objetivos de aprendizaje detallados basados en este programa de curso:\n{course_text}",
        "lecturas_sugeridas": f"Sugiere lecturas y recursos académicos basados en este programa de curso:\n{course_text}"
    }

    materials = {key: generate_content(prompt) for key, prompt in prompts.items()}
    
    return materials

# Prueba con el texto extraído
if __name__ == "__main__":
    file_path = "../PROGRAMA_DE_CURSO.pdf"  # Reemplaza con tu archivo
    course_text = extract_text(file_path)
    materials = generate_study_materials(course_text)

    for key, content in materials.items():
        print(f"\n=== {key.replace('_', ' ').title()} ===\n{content}...\n")



=== Notas Clase ===
## IA-101: Fundamentos de Inteligencia Artificial - Notas de Clase Detalladas

**Profesor:** Dr. Carlos Martínez
**Duración:** 16 semanas
**Créditos:** 4

**Descripción del Curso:** Este curso proporciona una introducción sólida a los conceptos y técnicas fundamentales de la Inteligencia Artificial (IA). Exploraremos la historia, los principios teóricos y las aplicaciones prácticas de la IA, centrándonos en algoritmos de búsqueda, aprendizaje automático (machine learning) y procesamiento del lenguaje natural (PLN). A través de proyectos prácticos, los estudiantes desarrollarán habilidades esenciales para implementar y evaluar soluciones de IA básicas.

**Objetivos del Curso:** Al finalizar este curso, los estudiantes serán capaces de:

*   Comprender los fundamentos teóricos que sustentan la IA, incluyendo su historia y evolución.
*   Aplicar diversos algoritmos de búsqueda y optimización para resolver problemas específicos.
*   Desarrollar e implementar modelos bá